# Full Model Inflation

### Initialization: libraries and initial settings

In [1]:
# ----- Load libraries -----

# Built-in
using Colors, CSV, Dates, DataFrames, Distributions, FileIO, JLD2, XLSX, KernelDensity, PlotlyJS;
pltjs = PlotlyJS

# Custom
include("./code/Metropolis-Within-Gibbs/MetropolisWithinGibbs.jl");
using Main.MetropolisWithinGibbs;

┌ Warning: Kaledio is not available on this system. Julia will be unable to produce any plots.
└ @ PlotlyBase C:\Users\wrc938\.julia\packages\PlotlyBase\NxSlF\src\kaleido.jl:58


In [2]:
# ----- Load results from output file -----

res           = load("$(pwd())/res_full_inf.jld2");
nDraws        = res["nDraws"];
burnin        = res["burnin"];
σʸ            = res["σʸ"]';
date          = res["date"];
data          = res["data"] .* σʸ';
distr_α       = res["distr_α"];
chain_θ_bound = res["chain_θ_bound"]; # d, Z, R, c, T, Q, λ, ρ, total
MNEMONIC      = res["MNEMONIC"];



data[ismissing.(data)] .= NaN;

par_ind  = res["par_ind"];


par_size = SizeParSsm(sum(par_ind.d),
                                 sum(sum(par_ind.Z)),
                                 sum(sum(par_ind.Z_plus)),
                                 sum(sum(par_ind.Z_minus)),
                                 sum(sum(par_ind.R)),
                                 sum(par_ind.c),
                                 sum(sum(par_ind.T)),
                                 sum(sum(par_ind.Q)),
                                 sum(sum(par_ind.Q_cov)), 
                                 sum(sum(par_ind.λ)),
                                 sum(sum(par_ind.ρ)),
                                 sum(par_ind.d) + sum(sum(par_ind.Z)) + sum(sum(par_ind.Z_plus)) +
                                    sum(sum(par_ind.Z_minus)) + sum(sum(par_ind.R)) +
                                    sum(par_ind.c) + sum(sum(par_ind.T)) + sum(sum(par_ind.Q)) +
                                    sum(sum(par_ind.Q_cov)) + sum(sum(par_ind.λ)) + sum(sum(par_ind.ρ)));





## Load States and data

In [3]:
# -----------------------------
# Data slices (as you set)
# -----------------------------
y         = data[:, 1]
e_data    = data[:, 2]
u_data    = data[:, 3]
π_data    = data[:, 4]
π_c_data  = data[:, 5]
uom_data  = data[:, 6]
spf_data  = data[:, 7]

# -----------------------------
# Posterior summaries (as you set)
# -----------------------------
α_med = mapslices(x -> quantile(x, 0.5),   distr_α, dims=3)[:,:,1]
α_05  = mapslices(x -> quantile(x, 0.025),  distr_α, dims=3)[:,:,1]
α_16  = mapslices(x -> quantile(x, 0.16),   distr_α, dims=3)[:,:,1]
α_84  = mapslices(x -> quantile(x, 0.84),   distr_α, dims=3)[:,:,1]
α_95  = mapslices(x -> quantile(x, 0.975),  distr_α, dims=3)[:,:,1]

# -----------------------------
# State indices (new model)
# Order in code:
# PC(cycle) [2], EP(cycle+trend) [3],
# y(ct) [3], e(ct) [3], u(ct) [3],
# π(cycle) [2], π^c(cycle) [2],
# UoM(ct) [3], SPF(ct) [3]
# -----------------------------
C_PC   = 1             # common business cycle (Ψ^e cos)
C_EP   = 3             # common inflation cycle (Ψ^π cos)
τ_π    = 5             # common inflation trend μ^π

C_y    = 6             # idio y cycle (cos)
τ_y    = 8             # idio y trend

C_e_id = 9             # idio employment cycle (cos)
τ_e    = 11            # idio employment trend

C_u    = 12            # idio unemployment cycle (cos)
τ_u    = 14            # idio unemployment trend

C_π    = 15            # idio headline inflation cycle (cos)
C_π_c  = 17            # idio core inflation cycle (cos)

C_uom  = 19            # idio UoM inflation cycle (cos)
τ_uom  = 21            # idio UoM inflation trend

C_spf  = 22            # idio SPF inflation cycle (cos)
τ_spf  = 24            # idio SPF inflation trend



24

In [4]:
# -----------------------------
# Plot styling
# -----------------------------
main_color = "#2A6EA6"               # median line
band68     = "rgba(42,110,166,0.30)" # inner band
band90     = "rgba(42,110,166,0.15)" # outer band
data_color = "#222222"               # raw data

# ============================= Business Cycle (Common PC) =============================
plt_cycle = Plot([
    scatter(x=date, y=α_05[C_PC,:], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_PC,:], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_PC,:], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_PC,:], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_PC,:], mode="lines", name="Cyclical",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Output Gap (Common Business Cycle)", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=true),
    yaxis=attr(title="pct.", gridcolor="rgba(0,0,0,0.08)", zeroline=true),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_cycle)

# ============================= Trend Output + Data =============================
plt_trend_y = Plot([
    scatter(x=date, y=α_05[τ_y,:] .* σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_y,:] .* σʸ[1], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[τ_y,:] .* σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_y,:] .* σʸ[1], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[τ_y,:] .* σʸ[1], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=y, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Trend Output", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend_y)


# ============================= Idiosyncratic Output Cycle =============================
plt_idio_y = Plot([
    scatter(x=date, y=α_05[C_y,:] .* σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_y,:] .* σʸ[1], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_y,:] .* σʸ[1], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_y,:] .* σʸ[1], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_y,:] .* σʸ[1], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Component of Output", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=true),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=true),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_y)


# ============================= Trend Employment + Data =============================
plt_trend_e = Plot([
    scatter(x=date, y=α_05[τ_e,:] .* σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_e,:] .* σʸ[2], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[τ_e,:] .* σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_e,:] .* σʸ[2], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[τ_e,:] .* σʸ[2], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=e_data, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Trend Employment", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend_e)

# ========================== Idiosyncratic Employment Cycle ==========================
plt_idio_e = Plot([
    scatter(x=date, y=α_05[C_e_id,:] .* σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_e_id,:] .* σʸ[2], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_e_id,:] .* σʸ[2], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_e_id,:] .* σʸ[2], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_e_id,:] .* σʸ[2], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Employment", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=true),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=true),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_e)

# ============================= Unemployment trend + Data =============================
plt_trend_u = Plot([
    scatter(x=date, y=α_05[τ_u,:] .* σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_u,:] .* σʸ[3], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[τ_u,:] .* σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_u,:] .* σʸ[3], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[τ_u,:] .* σʸ[3], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=u_data, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Unemployment", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend_u)

# ========================== Idiosyncratic Unemployment Cycle ==========================
plt_idio_u = Plot([
    scatter(x=date, y=α_05[C_u,:] .* σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_u,:] .* σʸ[3], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_u,:] .* σʸ[3], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_u,:] .* σʸ[3], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_u,:] .* σʸ[3], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Unemployment", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=true),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=true),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_u)

# ============================= Inflation trend + Data =============================
plt_trend_π = Plot([
    scatter(x=date, y=α_05[τ_π,:], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_π,:], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[τ_π,:], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_π,:], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[τ_π,:], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5)),
    scatter(x=date, y=π_data, mode="lines", name="Data",
            line=attr(color=data_color, width=1.4), opacity=0.9)
],
Layout(
    title="Inflation", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_trend_π)

# ========================== Common Inflation Cycle ==========================
plt_cycle_π = Plot([
    scatter(x=date, y=α_05[C_EP,:], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_EP,:], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_EP,:], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_EP,:], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_EP,:], mode="lines", name="Cyclical",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Common Inflation Cycle", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_cycle_π)

# ========================== Idiosyncratic Inflation Cycle ==========================
plt_idio_π = Plot([
    scatter(x=date, y=α_05[C_π,:] .* σʸ[4], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_π,:] .* σʸ[4], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_π,:] .* σʸ[4], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_π,:] .* σʸ[4], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_π,:] .* σʸ[4], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Inflation Cycle", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_π)

# ========================== Idiosyncratic Core Inflation Cycle ==========================
plt_idio_π_c = Plot([
    scatter(x=date, y=α_05[C_π_c,:] .* σʸ[5], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_π_c,:] .* σʸ[5], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_π_c,:] .* σʸ[5], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_π_c,:] .* σʸ[5], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_π_c,:] .* σʸ[5], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic Core Inflation Cycle", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_π_c)

# ========================== Idiosyncratic UoM Inflation Cycle  ==========================
plt_idio_uom = Plot([
    scatter(x=date, y=α_05[C_uom,:] .* σʸ[6], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_uom,:] .* σʸ[6], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_uom,:] .* σʸ[6], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_uom,:] .* σʸ[6], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_uom,:] .* σʸ[6], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic UoM Inflation Cycle", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_uom)

# ========================== Idiosyncratic UoM Trend  ==========================
plt_idio_uom_t = Plot([
    scatter(x=date, y=α_05[τ_uom,:] .* σʸ[6], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_uom,:] .* σʸ[6], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[τ_uom,:] .* σʸ[6], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_uom,:] .* σʸ[6], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[τ_uom,:] .* σʸ[6], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic UoM Trend", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_uom_t)

# ========================== Idiosyncratic SPF Inflation Cycle  ==========================
plt_idio_spf = Plot([
    scatter(x=date, y=α_05[C_spf,:] .* σʸ[7], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[C_spf,:] .* σʸ[7], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[C_spf,:] .* σʸ[7], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[C_spf,:] .* σʸ[7], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[C_spf,:] .* σʸ[7], mode="lines", name="Idiosyncratic",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic SPF Inflation Cycle", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))
display(plt_idio_spf)

# ========================== Idiosyncratic SPF Trend  ==========================
plt_idio_spf_t = Plot([
    scatter(x=date, y=α_05[τ_spf,:] .* σʸ[7], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_95[τ_spf,:] .* σʸ[7], mode="lines", name="90% CI",
            fill="tonexty", fillcolor=band90, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_16[τ_spf,:] .* σʸ[7], mode="lines",
            line=attr(color="rgba(0,0,0,0)"), hoverinfo="skip", showlegend=false),
    scatter(x=date, y=α_84[τ_spf,:] .* σʸ[7], mode="lines", name="68% CI",
            fill="tonexty", fillcolor=band68, line=attr(color="rgba(0,0,0,0)")),
    scatter(x=date, y=α_med[τ_spf,:] .* σʸ[7], mode="lines", name="Trend",
            line=attr(color=main_color, width=2.5))
],
Layout(
    title="Idiosyncratic SPF Trend", template="plotly_white", hovermode="x unified",
    width=900, height=420, margin=attr(l=60,r=20,t=60,b=55),
    xaxis=attr(title="Date", showgrid=true, zeroline=false),
    yaxis=attr(title="log of Bill. Chn. 2009\$", gridcolor="rgba(0,0,0,0.08)", zeroline=false),
    legend=attr(orientation="h", x=0.0, y=1.02, yanchor="bottom", bgcolor="rgba(0,0,0,0)")
))


data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, opacity, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, opacity, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



...

data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



data: [
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields hoverinfo, line, mode, showlegend, type, x, and y",
  "scatter with fields fill, fillcolor, line, mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, hovermode, legend, margin, template, title, width, xaxis, and yaxis"



### All posteriors in $\theta$

In [5]:


n_par, n_draws = size(chain_θ_bound)

for j in 1:n_par
    θj = vec(chain_θ_bound[j, :])

    # KDE
    k   = kde(θj)
    kx  = k.x
    kd  = k.density

    # Summary stats
    μ   = mean(θj)
    med = median(θj)

    xmin, xmax = minimum(kx), maximum(kx)
    pad = 0.05 * max(xmax - xmin, eps())
    xrng = [xmin - pad, xmax + pad]

    # For vertical line height
    y_max = maximum(kd) * 1.05

    # Traces (posterior, mean, median)
    trace_post   = pltjs.scatter(x=kx, y=kd, line=attr(width=1.4, color=c1), name="Posterior", showlegend=true)
    trace_mean   = pltjs.scatter(x=[μ, μ],   y=[0, y_max], mode="lines",
                                 line=attr(width=1.2, color="black", dash="dash"),
                                 name="Mean", showlegend=true)
    trace_median = pltjs.scatter(x=[med, med], y=[0, y_max], mode="lines",
                                 line=attr(width=1.2, color="gray", dash="dot"),
                                 name="Median", showlegend=true)

    # Layout matching your Phillips-curve style
    layout = pltjs.Layout(; 
        title = "Posterior density for θ[$j]",
        titlefont = attr(size=12),
        xaxis = attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false, range=xrng),
        yaxis = attr(tickfont_size=10, showgrid=true, linecolor="black", mirror=true, zeroline=false,
                     titlefont=attr(size=10), title="Density"),
    
    )

    fig = pltjs.plot([trace_post, trace_mean, trace_median], layout)

    # Same figure sizing & legend styling as your example
    fig.plot.layout["width"]  = 600
    fig.plot.layout["height"] = 400
    fig.plot.layout["margin"][:b] = 40
    fig.plot.layout["margin"][:t] = 40
    fig.plot.layout["margin"][:r] = 40
    fig.plot.layout["margin"][:l] = 40
    fig.plot.layout["legend"] = attr(orientation="h", y=-0.05, x=0.4, font=attr(size=10))

    display(fig)

    # Optional console printout
    # println("θ[$j]: mean=$(round(μ, digits=4)), median=$(round(med, digits=4))")

    # Optional: save
    # savefig(fig, "./img/theta_$(j).pdf", format="pdf")
end


UndefVarError: UndefVarError: `c1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.